In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from tqdm import tqdm 
import wandb
from torch.optim.lr_scheduler import CosineAnnealingLR
import sys

if bool(os.environ.get("KAGGLE_URL_BASE", "")):
  import sys
  # running on kaggle
  sys.path.insert(0, "/kaggle/input/hsm-source-files")
else:
  # running locally
  sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..", "..", "..")))


from src.utils.k_folds_creator import KFoldCreator
from src.utils.utils import get_models_save_path, set_seeds
from src.utils.constants import Constants 
from src.datasets.eeg_dataset import EEGDataset
from src.models.tcn import TCNModel

2025-10-21 21:21:10,022 :: root :: INFO :: Initialising Utils
2025-10-21 21:21:10,024 :: root :: INFO :: Initialising Datasets
2025-10-21 21:21:10,049 :: root :: INFO :: Initialising Models
wandb: Currently logged in as: maikotrede (hms-hslu-aicomp-hs25) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
wandb.login()

In [ ]:

class CFG:
    # Input Sequence Length :
    #   - Original Sampling Rate: 200 Hz
    #   - Downsample Factor: 2
    #   - Effective Sampling Rate: 200 Hz / 2 = 100 Hz
    #   - Clip Duration: 50 seconds
    #   - Required Input Length: 100 Hz * 50 s = 5000 timesteps

    # Receptive Field Calculation TCN:
    #   - The RF formula is: 1 + Sum[(kernel_size - 1) * dilation] for each layer.
    #   - With 8 layers, the dilations are: 1, 2, 4, 8, 16, 32, 64, 128.
    #   - Kernel Size: 21, so (kernel_size - 1) = 20.
    #   - RF = 1 + (20 * 1) + (20 * 2) + (20 * 4) + (20 * 8) + (20 * 16) + (20 * 32) + (20 * 64) + (20 * 128)
    #   - RF = 1 + 20 * (1 + 2 + 4 + 8 + 16 + 32 + 64 + 128)
    #   - RF = 1 + 20 * (255)
    #   - RF = 1 + 5100 = 5101 timesteps
    seed = 42
    n_splits = 5
    data_path = '../../../data/'

    model_name = 'TCN'
    num_tcn_channels = [64, 128, 128, 256, 256, 512, 512, 512]
    kernel_size = 21 
    dropout = 0.35
    target_size = 6


    original_sampling_rate = 200 # Hz
    sequence_duration = 50
    num_channels = 20 # Number of input EEG channels
    
    enable_downsampling = True
    downsample_factor = 3 # 2 would be optimal, but 3 is used to fit in memory. maybe if we can get a better GPU...


    batch_size = 32
    num_workers = 0
    epochs = 15
    lr = 1e-4

if CFG.enable_downsampling:
    CFG.sampling_rate = CFG.original_sampling_rate // CFG.downsample_factor
else:
    CFG.sampling_rate = CFG.original_sampling_rate

CFG.sequence_length = CFG.sequence_duration * CFG.sampling_rate

set_seeds(CFG.seed)
TARGETS = Constants.TARGETS

In [ ]:
def get_dataloaders(df, fold_id):
    train_df = df[df['fold'] != fold_id].reset_index(drop=True)
    valid_df = df[df['fold'] == fold_id].reset_index(drop=True)

    downsample_factor = CFG.downsample_factor if CFG.enable_downsampling else 1

    train_dataset = EEGDataset(
        df=train_df, data_path=CFG.data_path, mode='train', 
        downsample_factor=downsample_factor
    )
    valid_dataset = EEGDataset(
        df=valid_df, data_path=CFG.data_path, mode='valid', 
        downsample_factor=downsample_factor
    )

    train_loader = DataLoader(
        train_dataset, batch_size=CFG.batch_size, shuffle=True,
        num_workers=CFG.num_workers, pin_memory=True, drop_last=True
    )
    
    valid_loader = DataLoader(
        valid_dataset, batch_size=CFG.batch_size, shuffle=False,
        num_workers=CFG.num_workers, pin_memory=True, drop_last=False
    )
    
    return train_loader, valid_loader


In [4]:
def run_training(df, DATA_PREPARATION_VOTE_METHOD):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    
    all_oof_preds = []
    all_oof_labels = []

    for fold in range(CFG.n_splits):
        print(f"\n========== FOLD {fold} ==========")

        config = {
            "architecture": CFG.model_name, "tcn_channels": CFG.num_tcn_channels,
            "kernel_size": CFG.kernel_size, "dropout": CFG.dropout,
            "fold": fold, "features": "raw_eeg", "sequence_duration": f"{CFG.sequence_duration}s",
            "optimizer": "AdamW", "learning_rate": CFG.lr, "batch_size": CFG.batch_size,
            "epochs": CFG.epochs, "seed": CFG.seed, "Scheduler": "CosineAnnealingLR" 
        }

        wandb.init(
            project="hms-aicomp-tcn",
            name=f"tcn-raw-eeg-fold{fold}", 
            tags=['tcn-baseline', f'fold{fold}'],
            config=config
        )

        model = TCNModel(
            num_inputs=CFG.num_channels,
            num_outputs=CFG.target_size,
            channel_sizes=CFG.num_tcn_channels,
            kernel_size=CFG.kernel_size,
            dropout=CFG.dropout,
            causal=False,
            use_skip_connections=True
        )
        model.to(device)

        optimizer = torch.optim.AdamW(model.parameters(), lr=CFG.lr)
        scheduler = CosineAnnealingLR(optimizer, T_max=CFG.epochs)
        loss_fn = nn.KLDivLoss(reduction='batchmean')
        train_loader, valid_loader = get_dataloaders(df, fold)

        best_val_loss = float('inf')
        best_model_path = get_models_save_path() / "TCNModel" / DATA_PREPARATION_VOTE_METHOD / f'best_model_fold{fold}.pth'
        best_model_path.parent.mkdir(parents=True, exist_ok=True)
        
        for epoch in range(CFG.epochs):
            print(f"   --- Epoch {epoch+1}/{CFG.epochs} ---")
            
            model.train()
            train_loss = 0
            for signals, labels in tqdm(train_loader, desc="Training"):
                signals, labels = signals.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs = model(signals)
                log_probs = F.log_softmax(outputs, dim=1)
                loss = loss_fn(log_probs, labels)
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                optimizer.step()
                train_loss += loss.item() * signals.size(0)
            train_loss /= len(train_loader.dataset)

            model.eval()
            valid_loss = 0
            with torch.no_grad():
                for signals, labels in tqdm(valid_loader, desc="Validation"):
                    signals, labels = signals.to(device), labels.to(device)
                    outputs = model(signals)
                    log_probs = F.log_softmax(outputs, dim=1)
                    loss = loss_fn(log_probs, labels)
                    valid_loss += loss.item() * signals.size(0)
            valid_loss /= len(valid_loader.dataset)
            
            epoch_lr = optimizer.param_groups[0]['lr']
            print(f"   Epoch {epoch+1}: Train Loss = {train_loss:.4f}, Valid Loss = {valid_loss:.4f}, LR = {epoch_lr:.6f}")
            wandb.log({
                "epoch": epoch + 1, "train/epoch_loss": train_loss, "val/loss": valid_loss,
                "val/kl_div": valid_loss, "train/epoch_lr": epoch_lr
            })

            if valid_loss < best_val_loss:
                best_val_loss = valid_loss
                torch.save(model.state_dict(), best_model_path)
                print(f"   New best model saved with validation loss: {best_val_loss:.4f}")
            scheduler.step()

        print("   --- Generating OOF predictions for fold {fold} ---")
        if best_model_path.exists():
            model.load_state_dict(torch.load(best_model_path))
            model.eval()
            
            fold_oof_preds = []
            fold_oof_labels = []

            with torch.no_grad():
                for signals, labels in tqdm(valid_loader, desc=f"OOF Prediction Fold {fold}"):
                    signals = signals.to(device)
                    outputs = model(signals)
                    probs = F.softmax(outputs, dim=1).cpu()
                    
                    fold_oof_preds.append(probs)
                    fold_oof_labels.append(labels.cpu()) 
            
            all_oof_preds.append(torch.cat(fold_oof_preds).numpy())
            all_oof_labels.append(torch.cat(fold_oof_labels).numpy())
            
            print(f"   Finished OOF predictions for fold {fold}")
        else:
            print(f"   WARNING: No model file found at {best_model_path}. Skipping OOF for this fold.")

        wandb.summary['best_val_kl_div'] = best_val_loss
        artifact = wandb.Artifact(f'model-fold{fold}', type='model')
        artifact.add_file(best_model_path)
        wandb.log_artifact(artifact)
        wandb.finish()
        
    final_oof_preds = np.concatenate(all_oof_preds)
    final_oof_labels = np.concatenate(all_oof_labels)

    return final_oof_preds, final_oof_labels

In [ ]:
DATA_PREPARATION_VOTE_METHOD = "sum_and_normalize"

print("Preparing data and creating folds...")
df = pd.read_csv(CFG.data_path + 'processed_data_sum_votes_window.csv') 

label_map = {t: i for i, t in enumerate(TARGETS)}
df['expert_consensus'] = df[TARGETS].idxmax(axis=1)

print('Train shape:', df.shape)
print('Targets', list(TARGETS))

fold_creator = KFoldCreator(n_splits=CFG.n_splits, seed=CFG.seed)
df = fold_creator.create_folds(df, stratify_col='expert_consensus', group_col='patient_id')

print("Folds created. Value counts per fold:")
print(df['fold'].value_counts())

oof_preds, oof_labels = run_training(df, DATA_PREPARATION_VOTE_METHOD)

print("\nCalculating final OOF score...")
oof_preds_tensor = torch.tensor(oof_preds, dtype=torch.float32)
oof_labels_tensor = torch.tensor(oof_labels, dtype=torch.float32)

log_oof_preds_tensor = torch.log(oof_preds_tensor)

kl_loss_fn = nn.KLDivLoss(reduction='batchmean')
overall_oof_score = kl_loss_fn(log_oof_preds_tensor, oof_labels_tensor).item()

print(f"\nOverall OOF KL Score: {overall_oof_score:.4f}")

Preparing data and creating folds...
Train shape: (17089, 12)
Targets ['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']
Folds created. Value counts per fold:
fold
0    3741
1    3703
2    3527
4    3081
3    3037
Name: count, dtype: int64
Using device: cuda

========== FOLD 0 ==========


   --- Epoch 1/15 ---


Training:   0%|          | 0/417 [00:00<?, ?it/s]

Validation:   0%|          | 0/117 [00:00<?, ?it/s]

   Epoch 1: Train Loss = 1.2292, Valid Loss = 1.2785, LR = 0.000100
   New best model saved with validation loss: 1.2785
   --- Epoch 2/15 ---


...

   Epoch 15: Train Loss = 0.2500, Valid Loss = 1.1380, LR = 0.000001
   --- Generating OOF predictions for fold {fold} ---


OOF Prediction Fold 4:   0%|          | 0/97 [00:00<?, ?it/s]

   Finished OOF predictions for fold 4


epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
train/epoch_loss,█▇▅▅▄▄▃▃▂▂▂▁▁▁▁
train/epoch_lr,███▇▇▆▆▅▄▃▃▂▂▁▁
val/kl_div,█▄▄▂▂▁▁▂▂▃▄▆▆▅▅
val/loss,█▄▄▂▂▁▁▂▂▃▄▆▆▅▅
best_val_kl_div,0.95284
epoch,15
train/epoch_loss,0.24998
train/epoch_lr,0.0
val/kl_div,1.13796
val/loss,1.13796



Calculating final OOF score...

Overall OOF KL Score (comparable to XGBoost): 0.8738


In [6]:
print(f"\nOverall OOF KL Score: {overall_oof_score:.4f}")


Overall OOF KL Score: 0.8738
